In [1]:
import pandas as pd

In [5]:
import numpy as np
from sklearn.svm import SVC

In [15]:
data = pd.read_csv("svm-data.csv", header = None)

In [16]:
clf = SVC(random_state=42, C=100000, kernel = "linear")

In [17]:
target = data[data.columns[0]]

In [18]:
data = data[data.columns[1:]]

In [19]:
clf.fit(data, target)

SVC(C=100000, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.0, kernel='linear', max_iter=-1, probability=False,
  random_state=42, shrinking=True, tol=0.001, verbose=False)

In [20]:
clf.support_

array([3, 4, 9], dtype=int32)

In [21]:
clf.support_vectors_

array([[ 0.98,  0.68],
       [ 0.48,  0.39],
       [ 0.53,  0.53]])

In [22]:
data

,1,2
0,0.70,0.29
1,0.23,0.55
2,0.72,0.42
3,0.98,0.68
4,0.48,0.39
5,0.34,0.73
6,0.44,0.06
7,0.40,0.74
8,0.18,0.18
9,0.53,0.53


#Part #2

In [23]:
from sklearn import datasets

newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [24]:
target = newsgroups.target
data = newsgroups.data

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
data = tfidf.fit_transform(data)

In [30]:
from sklearn.cross_validation import KFold
from sklearn.grid_search import GridSearchCV

grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(target.shape[0], n_folds=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(data, target)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1786, n_folds=5, shuffle=True, random_state=241),
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='accuracy', verbose=0)

In [31]:
print gs.grid_scores_

[mean: 0.55263, std: 0.02812, params: {'C': 1.0000000000000001e-05}, mean: 0.55263, std: 0.02812, params: {'C': 0.0001}, mean: 0.55263, std: 0.02812, params: {'C': 0.001}, mean: 0.55263, std: 0.02812, params: {'C': 0.01}, mean: 0.95017, std: 0.00822, params: {'C': 0.10000000000000001}, mean: 0.99328, std: 0.00455, params: {'C': 1.0}, mean: 0.99328, std: 0.00455, params: {'C': 10.0}, mean: 0.99328, std: 0.00455, params: {'C': 100.0}, mean: 0.99328, std: 0.00455, params: {'C': 1000.0}, mean: 0.99328, std: 0.00455, params: {'C': 10000.0}, mean: 0.99328, std: 0.00455, params: {'C': 100000.0}]


In [37]:
reduce(lambda a,b : a if a.mean_validation_score > b.mean_validation_score else b, gs.grid_scores_)

mean: 0.99328, std: 0.00455, params: {'C': 100000.0}

In [ ]:
C = 100000

In [38]:
best = SVC(C=100000.0, kernel = 'linear')
best.fit(data, target)

SVC(C=100000.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.0, kernel='linear', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

In [80]:
from itertools import izip
from scipy.sparse import coo_matrix

m = coo_matrix(best.coef_)
tuples = izip(m.row, m.col, abs(m.data))
tt = sorted(tuples, key = lambda x: (x[0], x[2]),reverse=True)
top_words = [int(t[1]) for t in tt[:10]]
print top_words

[24019, 12871, 5088, 5093, 17802, 23673, 21850, 5776, 15606, 18430]


In [85]:
all_words = tfidf.get_feature_names()

In [87]:
words = [all_words[i] for i in top_words]

In [89]:
sorted(words)

[u'atheism',
 u'atheists',
 u'bible',
 u'god',
 u'keith',
 u'moon',
 u'nick',
 u'religion',
 u'sky',
 u'space']

#Part #3

In [122]:
data = pd.read_csv("data-logistic.csv", header = None)
data.head()

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [123]:
target = data[data.columns[0]]
data = data[data.columns[1:]]

print data.shape

(205, 2)


In [152]:
import math

def comp_br(w):
    result = 1.
    result -= 1./ (1. + math.exp(-1 * w))
    return result
    
    
def gradient_descent(target, data, l2_penalty, step):
    w1 = 0.
    w2 = 0.
    change = 1.0
    iterations = 0
    while change > 1e-5 and iterations < 10000:
        iterations += 1
        w1_new = w1
        w2_new = w2
        for i in range(data.shape[0]):
            dots = np.dot([w1, w2], [data[i,0], data[i,1]])
            w1_new += step * (1./data.shape[0])  * target[i] * data[i,0] * comp_br(target[i] * dots)
            w2_new += step * (1./data.shape[0])  * target[i] * data[i,1] * comp_br(target[i] * dots)
        w1_new -= step * l2_penalty * w1
        w2_new -= step * l2_penalty * w2
        
        change = math.sqrt((w1_new - w1)**2 + (w2_new - w2)**2)
        w1 = w1_new
        w2 = w2_new
        
    print "Iterations = {0}".format(iterations)
    return [w1, w2]
            


In [153]:
step = 0.1

In [154]:
l2_penalty = 10
print data.head()

          1         2
0 -0.663827 -0.138526
1  1.994596  2.468025
2 -1.247395  0.749425
3  2.309374  1.899836
4  0.849143  2.407750


In [155]:
w0 = gradient_descent(target, data.as_matrix(), 0., step)
w10 = gradient_descent(target, data.as_matrix(), 10., step)

Iterations = 244
Iterations = 8


In [156]:
print w0
print w10

[0.28781162047177705, 0.091983302159254141]
[0.028558754546234209, 0.024780137249735525]


In [160]:
#predict probs
pred0 = data.apply(lambda s: comp_br(np.dot(w0, s)), axis = 1)
pred10 = data.apply(lambda s: comp_br(np.dot(w10, s)), axis = 1)

In [167]:
print pred0[:5]
print pred10[:5]
print target[:5]

0    0.550774
1    0.309794
2    0.572016
3    0.301657
4    0.385595
dtype: float64
0    0.505597
1    0.470504
2    0.504263
3    0.471772
4    0.479034
dtype: float64
0   -1
1    1
2   -1
3    1
4    1
Name: 0, dtype: int64


In [162]:
from sklearn.metrics import roc_auc_score

In [168]:
score0 = roc_auc_score(target, 1 - pred0)
score10 = roc_auc_score(target, 1 - pred10)

In [169]:
print score0
print score10

0.926857142857
0.936285714286
